# vessel_seg workflow demo
以 Normal_1 为例，运行中心线树与分支张量流水线。确保激活 `conda activate vessel_seg`。

In [1]:
from pathlib import Path
root = Path('..').resolve()
vtp = root / 'ASOCA2020/Normal/Centerlines/Normal_1.vtp'
mask = root / 'ASOCA2020/Normal/Annotations/Normal_1.nrrd'
tree_json = root / 'outputs/normal1_tree.json'
branch_npz = root / 'outputs/normal1_branch_dataset.npz'
branch_dir = root / 'outputs/branches_normal1'
sim_dir = root / 'outputs/normal1_similarity'
print(vtp, mask)


/mnt/c/Users/chenyihao/Documents/GitHub/vessel_seg/ASOCA2020/Normal/Centerlines/Normal_1.vtp /mnt/c/Users/chenyihao/Documents/GitHub/vessel_seg/ASOCA2020/Normal/Annotations/Normal_1.nrrd


In [ ]:
import subprocess, sys
from pathlib import Path
root = Path('..').resolve()
case = 'Normal_1'
vtp = root / 'ASOCA2020/Normal/Centerlines/Normal_1.vtp'
mask = root / 'ASOCA2020/Normal/Annotations_nii/Normal_1.nii.gz'
tree_json = root / f'outputs/{case}/tree.json'
branch_npz = root / f'outputs/{case}/branch_dataset.npz'
branch_dir = root / f'outputs/{case}/branches'
sim_dir = root / f'outputs/{case}/similarity'
vis_dir = root / f'outputs/{case}/vis'
vis_dir.mkdir(parents=True, exist_ok=True)

def run(cmd):
    full_cmd = f"{sys.executable} {cmd}"
    print('>>>', full_cmd)
    subprocess.run(full_cmd, shell=True, check=True, cwd=root)

# 1) 建树
run(f"scripts/build_centerline_tree.py --vtp {vtp} --out {tree_json}")
# 2) 分支张量
run(f"scripts/build_branch_dataset.py --vtp {vtp} --mask {mask} --tree {tree_json} --out {branch_npz} --branch_dir {branch_dir} --start_offset 3 --case {case}")
# 3) 相似性 & PCA
run(f"scripts/branch_similarity.py --npz {branch_npz} --out_dir {sim_dir}")


>>> python scripts/build_centerline_tree.py --vtp /mnt/c/Users/chenyihao/Documents/GitHub/vessel_seg/ASOCA2020/Normal/Centerlines/Normal_1.vtp --out /mnt/c/Users/chenyihao/Documents/GitHub/vessel_seg/outputs/normal1_tree.json


python: can't open file '/mnt/c/Users/chenyihao/Documents/GitHub/vessel_seg/notebooks/scripts/build_centerline_tree.py': [Errno 2] No such file or directory


CalledProcessError: Command 'python scripts/build_centerline_tree.py --vtp /mnt/c/Users/chenyihao/Documents/GitHub/vessel_seg/ASOCA2020/Normal/Centerlines/Normal_1.vtp --out /mnt/c/Users/chenyihao/Documents/GitHub/vessel_seg/outputs/normal1_tree.json' returned non-zero exit status 2.

### 可视化示例
- 中心线 vs 分割表面
- 中心线 vs 分割骨架
- 树的锥子图
- 相似度热图


In [ ]:
from IPython.display import Image
run(f"scripts/plot_centerline_vs_gt.py --vtp {vtp} --mask {mask} --out {vis_dir/'centerline_vs_gt.png'} --case {case}")
Image(filename=vis_dir/'centerline_vs_gt.png', width=600)


In [ ]:
run(f"scripts/compare_centerline_vs_gt_centerline.py --vtp {vtp} --mask {mask} --out {vis_dir/'centerline_vs_gt_centerline.png'} --case {case}")
Image(filename=vis_dir/'centerline_vs_gt_centerline.png', width=600)


In [ ]:
run(f"scripts/plot_tree_fs_cones.py --vtp {vtp} --out {vis_dir/'tree_fs_cones.png'} --case {case}")
Image(filename=vis_dir/'tree_fs_cones.png', width=600)


In [ ]:
Image(filename=sim_dir/'distance_heatmap.png', width=500)
